In [ ]:
!pip install tqdm


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
!pip install openai

Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 389.6/389.6 kB 6.1 MB/s eta 0:00:00
     -------------------------------------- 434.9/434.9 kB 9.0 MB/s eta 0:00:00
     ---------------------------------------- 76.4/76.4 kB 4.4 MB/s eta 0:00:00
     ------------------------------------- 202.1/202.1 kB 12.8 MB/s eta 0:00:00
     ---------------------------------------- 78.6/78.6 kB ? eta 0:00:00
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
     ---------------------------------------- 1.9/1.9 MB 9.4 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.9.0
    Uninstalling typing_extensions-4.9.0:
      Successfully uninstalled typing_extensions-4.9.0


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
API_KEY = ""

In [ ]:
from openai import OpenAI
import json

client = OpenAI(api_key=API_KEY)

# Simulate the GPT-4-mini model processing
def gpt4_mini_model(title: str, content: str, tags: str, instructions: str) -> str:
    try:
        # Prepare the prompt
        prompt = f"{instructions}\nTitle: {title}\nContent: {content}\nTags: {list(tags.split('|'))}"

        # Call GPT-4-mini (use "gpt-4-0613" or similar identifier for smaller versions if available)
        response =  client.chat.completions.create(
            model="gpt-4o-mini",  # Ensure this is the correct identifier for GPT-4-mini if available
            messages=[
                {"role": "system", "content": "You are a helpful assistant of analyzing blog posts. You are featured in analyzing the texts, language expressions of the blog post."},
                {"role": "user", "content":  prompt }
            ]
        )

        # Extract the response text
        result = response.choices[0].message.content
        # print(result)

         # Extract JSON portion from the response
        start = result.find("{")
        end = result.rfind("}") + 1
        json_text = result[start:end]

        # Parse the JSON text
        parsed_data = json.loads(json_text)
        return parsed_data
    except Exception as e:
        print(f"Error in model call: {e}")
        return "Error processing content."

In [ ]:
instructions = '''네이버 블로그 포스팅을 분석해서 광고성 글의 패턴과 특징을 분석해서 수치화된 정보를 뽑아낼거야. 자연어를 분석해서 다음의 지표들을 JSON 형식으로만 추출해줘. 지표 중 키워드는 띄어쓰기가 없는 단어, 띄어쓰기가 포함된 구, 절을 모두 포함해.
    1. frequently_used_expressions: 자주 사용되는 어휘(표현) * 반복된 횟수의 곱
    2. keyword_ratio: 반복되는 키워드가 전체 Context에서 차지하는 비중(%)
    3. emotion_distribution: 글의 감정 분포 상태(%)
    4. tone: 어조 파악 - 매우 긍정, 긍정, 중립, 부정, 매우 부정 5단계 중 하나로 선정
    5. layout_structure: 주제와 글을 전개해나가는 레이아웃의 짜임새 여부 - 크게 4개 이상의 소분류가 확인되면 Yes, 아니면 No\n

    Answer:
    {
        "frequently_used_expressions": {
            "맛집": 7,
            "맛있었어요": 2,
            "정말": 1,
        },
        "keyword_ratio": [{
            "keyword": "잠수교집",
            "ratio": 4.5
        }, {
            "keyword": "여의도",
            "ratio": 2.73
        }],
        "emotion_distribution": {
            "positive": 78,
            "neutral": 22,
            "negative": 0
        },
        "tone": "중립",
        "layout_structure": "Yes"
    }
'''

In [ ]:
instructions_2 = '''네이버 블로그 포스팅을 분석해서 광고성 글의 패턴과 특징을 분석해서 수치화된 정보를 뽑아낼거야. 자연어를 분석해서 다음의 지표들을 JSON 형식으로만 추출해줘.\n
1. frequently_used_expressions: 자주 사용되는 어휘(표현) * 반복된 횟수
2. keyword_ratio: 반복되는 키워드(단어/구/어절 단위 키워드)가 전체 Context에서 차지하는 비중(%)
3. sentiment_distribution: 글의 감정 분포 상태(%)
4. tone: 어조 파악 - 매우 긍정, 긍정, 중립, 부정, 매우 부정 5단계 중 하나로 선정
5. has_negative_content: 부정적인 감정이나 내용 포함 여부 (부정적/아쉬움/한계/불편함 의견)
6. structure: 주제와 글을 전개해나가는 레이아웃의 짜임새 여부 - 크게 4개 이상의 소분류가 확인되면 Yes, 아니면 No\n
7. has_ad: 제품/서비스 홍보성 문구 등장 여부 ("최고의", "추천", "강추" 등)
8. has_brand: 업체명 또는 브랜드명을 반복 언급 여부
9. has_promotion: 할인/프로모션/이벤트 관련 문구 포함 여부
10. is_provided: 체험단/협찬/제공 문구 포함 여부
11. overaction: 과도한 긍정적 표현 표기 여부
12. price: 상세한 가격(메뉴) 정보 포함 여부
13. has_question: 리뷰 후 질문 유도 여부
14. text_count: 글자수 카운트
15. is_my_money: 내돈내산, 내 돈 주고 내가 산 등의 키워드 포함 여부\n
Answer:
{
    "frequently_used_expressions": {
        "맛집": 7,
        "맛있었어요": 2,
        "정말": 1,
    },
    "keyword_ratio": [{
        "keyword": "잠수교집",
        "ratio": 4.5
    }, {
        "keyword": "여의도",
        "ratio": 2.73
    }],
    "sentiment_distribution": {
        "positive": 78,
        "neutral": 22,
        "negative": 0
    },
    "has_negative_content": true,
    "tone": "중립",
    "structure": true,
    "has_ad": false,
    "has_brand": true,
    "has_promotion": false,
    "is_provided": true,
    "overaction": true,
    "price": false,
    "has_question": true,
    "text_count": 1329,
    "is_my_money": true
}

"내돈내산" 키워드 포함 여부
'''

In [ ]:
instructions_3 = '''네이버 블로그 게시글이 내돈내산(내 돈 주고 내가 산) 키워드를 포함하고 있는지 알려줘. 자연어를 분석해서 다음 지표를 JSON 형식으로만 추출해줘.\n
- is_my_money: 내돈내산, 내 돈 주고 내가 산 등의 키워드 포함 여부\n
Answer:
{
    "is_my_money": true
}
'''

In [ ]:
import json
from tqdm import tqdm

# Load JSON file
with open('./rawdata/CRAWLING_TABLE_202411132039.json', mode='r', encoding='utf-8') as file:
    data = json.load(file)

# Process each entry and store results in a new list
processed_data = []
for item in tqdm(data["CRAWLING_TABLE"], desc="Processing entries"):
    title = item.get("title", "")
    content = item.get("content", "")
    tags = item.get("tags", "")

    # Run the GPT-4-mini model on the title, content, and tags
    result = gpt4_mini_model(title, content, tags, instructions_3)

    # Append the result to the entry dictionary
    processed_entry = item.copy()
    processed_entry["output"] = result
    processed_data.append(processed_entry)

Processing entries: 100%|██████████| 300/300 [04:01<00:00,  1.24it/s]


In [ ]:
from datetime import datetime

# Get the current date and time for the filename
current_datetime = datetime.now().strftime("%Y%m%d_%H%M%S")
output_filename = f'processed_data_{current_datetime}.json'

# Save the processed data to a new JSON file with the current datetime in the filename
with open(output_filename, 'w', encoding='utf-8') as file:
    json.dump({"CRAWLING_TABLE": processed_data}, file, ensure_ascii=False, indent=4)

print(f"Processing complete. Results saved to '{output_filename}'.")

Processing complete. Results saved to 'processed_data_20241116_204729.json'.


In [ ]:
pip install --upgrade mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 16.1/16.1 MB 9.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import json

# Load JSON file
with open('./processed_data_20241113_211534.json', mode='r', encoding='utf-8') as file:
    result_data = json.load(file)

In [ ]:
# Load 내돈내산 JSON file
with open('./processed_data_20241116_204729.json', mode='r', encoding='utf-8') as file:
    my_money_data = json.load(file)

In [ ]:
for idx, item in enumerate(result_data['CRAWLING_TABLE']):
    my_money = my_money_data['CRAWLING_TABLE'][idx]['output']['is_my_money']
    result_data['CRAWLING_TABLE'][idx]['output']['is_my_money'] = my_money

from datetime import datetime

# Get the current date and time for the filename
current_datetime = datetime.now().strftime("%Y%m%d_%H%M%S")
output_filename = f'processed_data_combined_{current_datetime}.json'

# Save the processed data to a new JSON file with the current datetime in the filename
with open(output_filename, 'w', encoding='utf-8') as file:
    json.dump({"CRAWLING_TABLE": result_data}, file, ensure_ascii=False, indent=4)

In [ ]:
sentiment_map = {
    "매우 긍정": 4,
    "긍정": 3,
    "중립": 2,
    "부정": 1,
    "매우 부정": 0
}

def convert_sentiment_to_numeric(sentiment):
    return sentiment_map.get(sentiment, None)

### MASTER DATA 만들기

In [ ]:
# Load 라벨링 JSON file
with open('./rawdata/labeling_data_20241116.json', mode='r', encoding='utf-8') as file:
    labeling_data = json.load(file)

In [ ]:
# Load 크롤링 JSON file
with open('./rawdata/CRAWLING_TABLE_202411162326.json', mode='r', encoding='utf-8') as file:
    master_data = json.load(file)

In [ ]:
master_data['CRAWLING_TABLE']

[{'id': 1,
  'link': 'https://blog.naver.com/PostView.naver?blogId=ydh77&logNo=223642562853&redirect=Dlog&widgetTypeCall=true&noTrackingCode=true&directAccess=false',
  'title': '여의도 직장인 대존맛탱 미나리생삼겹살 회식 [잠수교집 여의도 직영점] 내돈내산 솔직후기',
  'content': '여의도 직장인 대존맛탱 미나리생삼겹살\n맛집 인정\n잠수교집 여의도 직영점\n내돈내산 솔직후기\n레쓰고\n회사에 바빳던 일들이 거의 마무리가 되어서\n팀 회식을 하기로 했다\n\u200b\n내가 근무하는 회사는 저녁회식은 절대 안하는 곳이라\n항상 회식은 점심으로\n\u200b\n\u200b\n\u200b\n\u200b\n\u200b\n\u200b\n\u200b\n아쥬 만족한다\n\u200b\n\u200b\n어떤곳으로 가야지 기깔나는 회식을 했다고 할까 하다가\n눈에 들어온 기깔나는 고기집으로 선택함\n\u200b\n\u200b\n소신발언: 대존맛 맛집임\n\u200b\n잠수교집 여의도 직영점\n영업시간 11:30~22:00\n라스트오더 20:50\n단체, 무선인터넷, 화장실(남/녀 구분)\nT.0507-1359-0434\n잠수교집 여의도 직영점\n서울특별시 영등포구 여의나루로 117 지하 제6호 잠수교집\n더현대서울 근처, 여의나루역 근처에 위치하고있다\n그 유명한 공작아파트 상가쪽에 위치\n\u200b\n이곳 상가에 식당들이 꽤 많이 있어서\n여의도직장인들이 점심 먹으러 많이 온다\n\u200b\n\u200b\n\u200b\n\u200b\n힙하다 힙해\n\u200b\n상가건물 지하1층에 위치하고 있는 잠수교집\n네온사인간판이 저녁 회식 감성 낭낭하다\n하지만 우린 점심 회식\n\u200b\n\u200b\n\u200b\n\u200b\n\u200b\n\u200b\n오픈하자마자 오픈런 갈겼다\n솔직히 들어오면서도 \n\u200b\n\

In [ ]:
# Load GPT JSON file
with open('./rawdata/GPT_TABLE_202411162334.json', mode='r', encoding='utf-8') as file:
    gpt_data = json.load(file)

In [ ]:
gpt_data['GPT_TABLE']

[{'id': 1,
  'freq_used_expressions': '{"대존맛": 5, "여의도": 5, "삼겹살": 4, "미나리": 4, "맛있다": 4, "맛집": 4, "회식": 3, "냉삼": 3, "정말": 2}',
  'freq_used_cnt': 9,
  'freq_used_max': 5,
  'freq_used_sum': 34,
  'keyword_ratio': '[{"keyword": "여의도", "ratio": 3.76}, {"keyword": "잠수교집", "ratio": 2.56}, {"keyword": "삼겹살", "ratio": 2.18}]',
  'keyword_ratio_max': 3.76,
  'keyword_ratio_sum': 8.5,
  'sentiment_positive': 85,
  'sentiment_neutral': 15,
  'sentiment_negative': 0,
  'has_negative_content': 0,
  'tone': '매우 긍정',
  'structure': 0,
  'has_ad': 0,
  'has_brand': 1,
  'has_promotion': 0,
  'is_provided': 1,
  'overaction': 1,
  'price': 0,
  'has_question': 0,
  'text_count': 1878,
  'is_my_money': 1},
 {'id': 2,
  'freq_used_expressions': '{"냉삼 맛집": 6, "잠수교집": 5, "여의도": 5, "부산": 4, "맛있었어요": 3}',
  'freq_used_cnt': 5,
  'freq_used_max': 6,
  'freq_used_sum': 23,
  'keyword_ratio': '[{"keyword": "냉삼 맛집", "ratio": 6.6}, {"keyword": "잠수교집", "ratio": 5.5}, {"keyword": "여의도", "ratio": 5.5}]',
  'keywo

In [ ]:
# JSON 생성용
json_data = [];
try:
    for idx, item in enumerate(master_data['CRAWLING_TABLE']):
        data = gpt_data['GPT_TABLE'][idx]
        label = labeling_data['CRAWLING_LABEL'][idx]

        obj = {
            'id': item['id'],
            'has_map': item['has_map'],
            'image_cnt': item['image_cnt'],
            'visit_no': item['visit_no'],
            'tag_cnt': item['tag_cnt'],
            'freq_used_cnt': data['freq_used_cnt'],
            'freq_used_max': data['freq_used_max'],
            'freq_used_sum': data['freq_used_sum'],
            'keyword_ratio_max': data['keyword_ratio_max'],
            'keyword_ratio_sum': data['keyword_ratio_sum'],
            'sentiment_positive': data['sentiment_positive'],
            'sentiment_neutral': data['sentiment_neutral'],
            'sentiment_negative': data['sentiment_negative'],
            'has_negative_content': data['has_negative_content'],
            'tone': convert_sentiment_to_numeric(data['tone']),
            'structure': data['structure'],
            'has_ad': data['has_ad'],
            'has_brand': data['has_brand'],
            'has_promotion': data['has_promotion'],
            'is_provided': data['is_provided'],
            'overaction': data['overaction'],
            'price': data['price'],
            'has_question': data['has_question'],
            'text_count': data['text_count'],
            'is_my_money': data['is_my_money'],
            'ad_yn': 1 if label['ad_yn'] == 'Y' else 0
        }
        json_data.append(obj)

    # Get the current date and time for the filename
    current_datetime = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_filename = f'master_data_{current_datetime}.json'

    # Save the processed data to a new JSON file with the current datetime in the filename
    with open(output_filename, 'w', encoding='utf-8') as file:
        json.dump({"data": json_data}, file, ensure_ascii=False, indent=4)
except Exception as e:
    print(f"Error: {e}")

In [ ]:
# DB 저장용
import mysql.connector

try:
    # MySQL 데이터베이스 연결 설정
    connection = mysql.connector.connect(
        host="db-crawler-test.cy62je3vyh4c.ap-northeast-2.rds.amazonaws.com",        # 호스트 이름 (예: "localhost")
        user="2024511049",     # 사용자 이름
        password="shBUOnn5lKqJ7Ep", # 비밀번호
        database="blog"  # 사용할 데이터베이스 이름
    )

    # 커서 생성
    cursor = connection.cursor()

    for idx, item in enumerate(result_data['CRAWLING_TABLE']):
        data = item['output']
        # 변환 및 계산 작업
        freq_used_expressions_json = json.dumps(data['frequently_used_expressions'], ensure_ascii=False)
        freq_used_cnt = len(data['frequently_used_expressions'])
        freq_used_max = max(data['frequently_used_expressions'].values())
        freq_used_sum = sum(data['frequently_used_expressions'].values())

        keyword_ratio_json = json.dumps(data['keyword_ratio'], ensure_ascii=False)
        keyword_ratio_max = max([item['ratio'] for item in data['keyword_ratio']])
        keyword_ratio_sum = sum([item['ratio'] for item in data['keyword_ratio']])

        my_money = my_money_data['CRAWLING_TABLE'][idx]['output']['is_my_money']

        # INSERT 쿼리 및 데이터
        insert_query = """
            INSERT INTO GPT_TABLE (
                id, freq_used_expressions, freq_used_cnt, freq_used_max, freq_used_sum,
                keyword_ratio, keyword_ratio_max, keyword_ratio_sum,
                sentiment_positive, sentiment_neutral, sentiment_negative,
                has_negative_content, tone, structure, has_ad, has_brand,
                has_promotion, is_provided, overaction, price, has_question,
                text_count, is_my_money
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """

        insert_data = (
            item['id'],
            freq_used_expressions_json,
            freq_used_cnt,
            freq_used_max,
            freq_used_sum,
            keyword_ratio_json,
            keyword_ratio_max,
            keyword_ratio_sum,
            data['sentiment_distribution']['positive'],
            data['sentiment_distribution']['neutral'],
            data['sentiment_distribution']['negative'],
            data['has_negative_content'],
            data['tone'],
            data['structure'],
            data['has_ad'],
            data['has_brand'],
            data['has_promotion'],
            data['is_provided'],
            data['overaction'],
            data['price'],
            data['has_question'],
            data['text_count'],
            my_money
        )
        # 데이터 삽입 실행
        cursor.execute(insert_query, insert_data)
        connection.commit()  # 변경 사항 저장
        print(f"{data} 데이터가 성공적으로 삽입되었습니다.", id)
except mysql.connector.Error as error:
    print(f"데이터 삽입 중 오류 발생: {error}")
    connection.rollback()  # 오류 발생 시 변경 사항 되돌리기
finally:
    if connection.is_connected():
        # 연결 종료
        cursor.close()
        connection.close()

{'frequently_used_expressions': {'대존맛': 5, '여의도': 5, '삼겹살': 4, '미나리': 4, '맛있다': 4, '맛집': 4, '회식': 3, '냉삼': 3, '정말': 2}, 'keyword_ratio': [{'keyword': '여의도', 'ratio': 3.76}, {'keyword': '잠수교집', 'ratio': 2.56}, {'keyword': '삼겹살', 'ratio': 2.18}], 'sentiment_distribution': {'positive': 85, 'neutral': 15, 'negative': 0}, 'has_negative_content': False, 'tone': '매우 긍정', 'structure': 'No', 'has_ad': False, 'has_brand': True, 'has_promotion': False, 'is_provided': True, 'overaction': True, 'price': False, 'has_question': False, 'text_count': 1878, 'is_my_money': True} 데이터가 성공적으로 삽입되었습니다. <built-in function id>
{'frequently_used_expressions': {'냉삼 맛집': 6, '잠수교집': 5, '여의도': 5, '부산': 4, '맛있었어요': 3}, 'keyword_ratio': [{'keyword': '냉삼 맛집', 'ratio': 6.6}, {'keyword': '잠수교집', 'ratio': 5.5}, {'keyword': '여의도', 'ratio': 5.5}], 'sentiment_distribution': {'positive': 70, 'neutral': 25, 'negative': 5}, 'tone': '긍정', 'has_negative_content': True, 'structure': True, 'has_ad': False, 'has_brand': True, 'has_